In [1]:
from config import *
from dataset import ClassificationDataset

import functools

internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbr"
fs_method, fs_ratio = "mireg", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model()
Valid:         AUC: 0.7911 | AUPR: 0.2376 | Acc: 86.68%
best model()
Valid:         AUC: 0.7911 | AUPR: 0.2376 | Acc: 86.68%
Internal test: AUC: 0.7973 | AUPR: 0.2614 | Acc: 87.10%
External test: AUC: 0.7820 | AUPR: 0.2292 | Acc: 85.69%


In [3]:
from sklearn.naive_bayes import ComplementNB

model_fn = ComplementNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model()
Valid:         AUC: 0.7701 | AUPR: 0.1962 | Acc: 83.05%
best model()
Valid:         AUC: 0.7701 | AUPR: 0.1962 | Acc: 83.05%
Internal test: AUC: 0.7720 | AUPR: 0.2090 | Acc: 82.70%
External test: AUC: 0.7621 | AUPR: 0.1918 | Acc: 81.82%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto',
    n_neighbors=1, )
param_grid_list = [{
    'p': [1, 2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(p=1)
Valid:         AUC: 0.6963 | AUPR: 0.2599 | Acc: 93.15%
model(p=2)
Valid:         AUC: 0.6871 | AUPR: 0.2316 | Acc: 92.46%
best model(p=1)
Valid:         AUC: 0.6963 | AUPR: 0.2599 | Acc: 93.15%
Internal test: AUC: 0.7055 | AUPR: 0.2802 | Acc: 92.82%
External test: AUC: 0.7080 | AUPR: 0.2876 | Acc: 93.29%


In [5]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)

model(C=1.0)
Valid:         AUC: 0.7262 | AUPR: 0.3985 | Acc: 95.28%
model(C=10.0)
Valid:         AUC: 0.7329 | AUPR: 0.4083 | Acc: 95.35%
model(C=100.0)
Valid:         AUC: 0.7325 | AUPR: 0.4069 | Acc: 95.33%
best model(C=10.0)
Valid:         AUC: 0.7329 | AUPR: 0.4083 | Acc: 95.35%
Internal test: AUC: 0.7330 | AUPR: 0.4308 | Acc: 95.21%
External test: AUC: 0.7202 | AUPR: 0.3855 | Acc: 94.94%


In [6]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, 
    class_weight='balanced', kernel='linear',
    )
param_grid_list = [{
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(C=0.01)
Valid:         AUC: 0.8304 | AUPR: 0.2603 | Acc: 86.00%
model(C=0.1)
Valid:         AUC: 0.8347 | AUPR: 0.2616 | Acc: 85.78%
model(C=1)
Valid:         AUC: 0.8335 | AUPR: 0.2625 | Acc: 85.96%
best model(C=0.1)
Valid:         AUC: 0.8347 | AUPR: 0.2616 | Acc: 85.78%
Internal test: AUC: 0.8448 | AUPR: 0.2989 | Acc: 86.98%
External test: AUC: 0.8353 | AUPR: 0.2687 | Acc: 85.55%


In [7]:
# from sklearn.neural_network import MLPClassifier

# model_fn = functools.partial(MLPClassifier, random_state=SEED, max_iter=1000)
# param_grid_list = [{
#     'hidden_layer_sizes': [(100,), (50, 50), (50, 100, 50)],
#     'activation': ['logistic', 'relu'],
#     'solver': ['adam', 'sgd'],
#     'alpha': [0.0001, 0.001, 0.01],
# }]
# best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
# metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

In [8]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(criterion=gini)
Valid:         AUC: 0.8140 | AUPR: 0.2472 | Acc: 85.82%
model(criterion=log_loss)
Valid:         AUC: 0.8130 | AUPR: 0.2672 | Acc: 87.61%
model(criterion=entropy)
Valid:         AUC: 0.8130 | AUPR: 0.2672 | Acc: 87.61%
best model(criterion=gini)
Valid:         AUC: 0.8140 | AUPR: 0.2472 | Acc: 85.82%
Internal test: AUC: 0.8106 | AUPR: 0.2616 | Acc: 86.06%
External test: AUC: 0.7993 | AUPR: 0.2381 | Acc: 85.19%


In [9]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.8245 | AUPR: 0.3069 | Acc: 90.04%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.8288 | AUPR: 0.3132 | Acc: 90.18%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.8270 | AUPR: 0.3120 | Acc: 90.21%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.8257 | AUPR: 0.3081 | Acc: 90.07%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.8267 | AUPR: 0.3095 | Acc: 90.09%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.8287 | AUPR: 0.3154 | Acc: 90.32%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.8257 | AUPR: 0.3081 | Acc: 90.07%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.8267 | AUPR: 0.3095 | Acc: 90.09%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.8287 | AUPR: 0.3154 | Acc: 90.32%
best model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.8288 | AUPR: 0.3132 | Acc: 90.18%
Internal test: AU

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(criterion=friedman_mse)
Valid:         AUC: 0.7346 | AUPR: 0.3959 | Acc: 95.19%
model(criterion=squared_error)
Valid:         AUC: 0.7332 | AUPR: 0.3927 | Acc: 95.16%
best model(criterion=friedman_mse)
Valid:         AUC: 0.7346 | AUPR: 0.3959 | Acc: 95.19%
Internal test: AUC: 0.7353 | AUPR: 0.4218 | Acc: 95.09%
External test: AUC: 0.7345 | AUPR: 0.3982 | Acc: 94.98%


In [11]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)

model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.7393 | AUPR: 0.4038 | Acc: 95.25%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.7401 | AUPR: 0.4080 | Acc: 95.30%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.7361 | AUPR: 0.4043 | Acc: 95.28%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.7401 | AUPR: 0.4080 | Acc: 95.30%
Internal test: AUC: 0.7422 | AUPR: 0.4414 | Acc: 95.28%
External test: AUC: 0.7325 | AUPR: 0.3977 | Acc: 94.99%
